In [3]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

# ここではモデルを訓練しないため，訓練関連の演算は全て無効にする
K.set_learning_phase(0)

# Inception_v3ネットワークを畳み込みベースなしで構築する
# このモデルは学習済みのImageNetの重み付き読み込まれる
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

 6987776/87910968 [=>............................] - ETA: 1:12:20

In [1]:
# 層の名前を係数にマッピングするディクショナリ．この係数は最大かの対象と
# なる損失値にその層の活性化はどれくらい貢献するかを表す．これらの層の
# 名前は組み込みのInceptionV3アプリケーションにハードコーディングされていることに注意
# 全ての層の名前はmodel.summary()を使って確認できる

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [2]:
# 最大かの対象となる損失値を定義

# 層の名前を層のインスタンスにマッピングするディクショナリを作成
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# 損失値を定義
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]

    # 層の出力を取得
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation)), 'float32')

    # 層の特徴量のL2ノルムをlossに加算
    # 非境界ピクセルののみをlossに適用することで，周辺効果を回避
    loss += coeff * K.sum(K.square(activation[:, 2: -2, -2, :])) / scaling

NameError: name 'model' is not defined

In [ ]:
# 勾配上昇法のプロセス
# 生成された画像（ドリーム）を保持するテンソル
dream = model.input

# ドリームの損失関数の勾配を計算
grads = K.gradients(loss, dream)[0]

# 勾配を正規化（重要）
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 入力画像に基づいて損失と勾配の値を取得するkeras関数を設定
output = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 勾配上昇法を指定された回数に渡って実行する関数
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss Value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [ ]:
# 補助関数
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x
